In [1]:
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append("../../")
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import argparse

In [2]:
def customOneHotEncoder(data):
    dataAdjust = data.ljust(200,'0')[:200] # padding if not of length and adjusting the data lenght to get a 200x39 input matrix
    # define universe of possible input values
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyz,._-/+'
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in dataAdjust]
    #print(integer_encoded)
    # one hot encode
    onehot_encoded = list()
    for i, value in enumerate(integer_encoded):
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    #print(onehot_encoded) # the real encoding
    return onehot_encoded

# takes a .csv filename
def dataPreprocessing(fileName):
    df = pd.read_csv(fileName, header = None)
    
    #prepare the imput data
    xString = df.iloc[:,:43].to_string(header=False, index=False, index_names = False).split('\n')
    xList = [','.join(ele.split()) for ele in xString] # gives comma separated strings for each row of DataFrame
    xData = []
    for string in xList:
        stringLower = string.lower()
        oneHot = customOneHotEncoder(stringLower)
        xData.append(oneHot)
    xMid = np.array(xData)
    xArray = xMid.transpose(0,2,1) # convert xMid's dim (size, 200, 39) to (size, 39, 200)
    
    #prepare the label data
    df[43] = np.where(pd.isnull(df[47]), 'Normal', 'attack') # replacing anything except 'normal' with 'attack'
    Ydf = df[43]
    #labelName = Ydf.unique().tolist().sort() # sorted 38 label names
    #yArray = Ydf.str.get_dummies().to_numpy() # ndarray of shape(rows/lines, 38)
    yArray = Ydf.to_numpy()
    
    assert xArray.shape[0] == yArray.shape[0], 'unequal input and label sample size'
    
    
    return xArray, yArray # return processed array of input and label

In [3]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

class NSLKDDDataset(Dataset):
    def __init__(self, fileName):
        self.data = pd.read_csv(fileName, header = None)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # prepare x data
        string = list(','.join('%s' %x for x in y) for y in self.data.iloc[[idx], :47].values)
        stringLower = string[0].lower()
        xData = customOneHotEncoder(stringLower) # Dim (200, 39)
        xMid = np.array(xData)
        xArray = xMid.transpose() # should be now (39, 200)
        
        # prepate y data
        #self.data.iloc[idx, 41] = np.where(self.data.iloc[idx, 41]=='normal', 0, 1) # replacing normals with 0 and anything else with 1
        yArray = np.where(pd.isnull(self.data.iloc[idx, 47]), 0, 1)
        
        #yArray = Ydf.to_numpy()
    
        #assert xArray.shape == yArray.shape, 'unequal input and label sample size'
        
        return torch.from_numpy(xArray), torch.from_numpy(yArray) # returns torch tensor of x and y

In [4]:
params = {'batch_size': 64, 'shuffle': True}
fileNameTrain = 'UNSW_NB15_training-set.csv'
fileNameTest = 'UNSW_NB15_testing-set.csv'
datasetTrain = NSLKDDDataset(fileNameTrain)
datasetTest = NSLKDDDataset(fileNameTest)
dataGeneratorTrain = DataLoader(datasetTrain, **params)
dataGeneratorTest = DataLoader(datasetTest, **params)


In [4]:
params = {'batch_size': 64, 'shuffle': True}
filedataset = 'UNSW-NB15_1.csv'
dataset = NSLKDDDataset(filedataset)
total_length = dataset.__len__()
train_length = int(0.6 * total_length)
test_length = int(0.2 * total_length)
val_length = total_length - train_length - test_length
datasetTrain, datasetVal, datasetTest = torch.utils.data.random_split(dataset, [train_length, val_length, test_length])
dataGeneratorTrain = DataLoader(datasetTrain, **params)
dataGeneratorTest = DataLoader(datasetTest, **params)
dataGeneratorVal = DataLoader(datasetVal, **params)

C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (1,3,47) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
datasetTrain.__len__()

420000

In [7]:
root = './data/mnist'
batch_size = 64
n_classes = 2
input_channels = 42
seq_length = int(200)
epochs = 10
steps = 0

In [8]:
#train_loader, test_loader = data_generator(root, batch_size)

permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
channel_sizes = [32] * 6 #hidden nodes times levels 
kernel_size = 5
nheads = 1

model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=0.25)
#input_size, output_size, num_channels, 
#kernel_size, dropout, num_sub_blocks, temp_attn, nheads, en_res,                              num_sub_blocks
#conv, key_size
#
# nheads, key_size


lr = 1e-5
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)

In [9]:
def train(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(dataGeneratorTrain):
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        optimizer.zero_grad()
        #print(data.shape)
        data = data.view(-1, input_channels, seq_length)
        data, target = Variable(data), Variable(target)
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        #print(target)
        optimizer.zero_grad()
        #print(data[0])
        data = data.type(torch.FloatTensor)
        output = model(data)
        #print(output.shape)
        target = target.type(torch.LongTensor)
        loss1 = torch.nn.CrossEntropyLoss()
        loss = F.nll_loss(output, target) # negative log likelihood
        loss = loss1(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss
        steps += seq_length
        #print(batch_idx)
        if batch_idx > 0 and batch_idx % 800 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(dataGeneratorTrain.dataset),
                100. * batch_idx / len(dataGeneratorTrain), train_loss.item()/100, steps))
            train_loss = 0

In [10]:
def test():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorTest:
            model.eval()
            #print('checkpoint 1')
            #data = data.view(-1, input_channels, seq_length)
            #print('checkpoint 2')
            data = data.type(torch.FloatTensor)
            #print('checkpoint 3')
            target = target.type(torch.LongTensor)
            #print('checkpoint 4')
            data, target = Variable(data, volatile=True), Variable(target)
            #print('checkpoint 5')
            output = model(data)
            #print('checkpoint 6')
            #loss1 = torch.nn.CrossEntropyLoss()
            test_loss += F.nll_loss(output, target, size_average=False).item()
            #print('checkpoint 7')
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            #print('checkpoint 8')
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            #print('checkpoint 9')

        test_loss /= len(dataGeneratorTest.dataset)
        #print('checkpoint 10')
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorTest.dataset),
            100. * correct / len(dataGeneratorTest.dataset)))
        return test_loss

In [11]:
if __name__ == "__main__":
    for epoch in range(1, epochs+1):
        train(epoch)
        test()
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

Train Epoch: 1 [51200/420000 (12%)]	Loss: 2.416590	Steps: 160200
Train Epoch: 1 [102400/420000 (24%)]	Loss: 1.124084	Steps: 320200
Train Epoch: 1 [153600/420000 (37%)]	Loss: 1.088910	Steps: 480200
Train Epoch: 1 [204800/420000 (49%)]	Loss: 1.078178	Steps: 640200
Train Epoch: 1 [256000/420000 (61%)]	Loss: 1.063734	Steps: 800200
Train Epoch: 1 [307200/420000 (73%)]	Loss: 0.998640	Steps: 960200
Train Epoch: 1 [358400/420000 (85%)]	Loss: 0.944188	Steps: 1120200
Train Epoch: 1 [409600/420000 (98%)]	Loss: 0.834705	Steps: 1280200


C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0814, Accuracy: 135506/140000 (97%)

Train Epoch: 2 [51200/420000 (12%)]	Loss: 0.553623	Steps: 1472800
Train Epoch: 2 [102400/420000 (24%)]	Loss: 0.314074	Steps: 1632800
Train Epoch: 2 [153600/420000 (37%)]	Loss: 0.211456	Steps: 1792800
Train Epoch: 2 [204800/420000 (49%)]	Loss: 0.170805	Steps: 1952800
Train Epoch: 2 [256000/420000 (61%)]	Loss: 0.168684	Steps: 2112800
Train Epoch: 2 [307200/420000 (73%)]	Loss: 0.164602	Steps: 2272800
Train Epoch: 2 [358400/420000 (85%)]	Loss: 0.154673	Steps: 2432800
Train Epoch: 2 [409600/420000 (98%)]	Loss: 0.164506	Steps: 2592800

Test set: Average loss: 0.0183, Accuracy: 139021/140000 (99%)

Train Epoch: 3 [51200/420000 (12%)]	Loss: 0.152688	Steps: 2785400
Train Epoch: 3 [102400/420000 (24%)]	Loss: 0.146438	Steps: 2945400
Train Epoch: 3 [153600/420000 (37%)]	Loss: 0.155771	Steps: 3105400
Train Epoch: 3 [204800/420000 (49%)]	Loss: 0.157154	Steps: 3265400
Train Epoch: 3 [256000/420000 (61%)]	Loss: 0.143733	Steps: 3425400
Tra

KeyboardInterrupt: 

In [12]:
#NEED TO DO THIS ONE
from sklearn.metrics import classification_report
def get_metrics(dataLoader):
    total = 0
    correct = 0
    precision = 0
    recall = 0
    f1_score = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in dataLoader: # just 1 batch
            model.eval()
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            #print(predicted)
            total+=target.size(0)
            correct+=(predicted == target).sum().item()
            report = classification_report(target, predicted, output_dict=True)
            precision += report['macro avg']['precision']
            recall += report['macro avg']['recall']
            f1_score += report['macro avg']['f1-score']
            accuracy += report['accuracy']
            print(report)
    print("Precision: {}, Recall: {}, F1-Score: {}, Accuracy: {}, AccuracyCust: {}".format(precision, recall, f1_score, accuracy, correct/total))
    

In [ ]:
#for now don't do thisone
#fileNameVal = 'UNSW_NB15_testing-set.csv'
#datasetVal = NSLKDDDataset(fileNameVal)
params = {'batch_size': 22544, 'shuffle': True}
dataGeneratorVal = DataLoader(datasetVal, **params)
#RuntimeError: expected scalar type Int but found Float
#get_metrics(dataGeneratorVal)
def val():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorVal:
            model.eval()
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            #loss1 = torch.nn.CrossEntropyLoss()
            #test_loss += F.nll_loss(output, target, size_average=False).item()
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            print(pred)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorVal.dataset),
            100. * correct / len(dataGeneratorVal.dataset)))
        return test_loss

val()


In [13]:
#do this one last
params = {'batch_size': datasetVal.__len__(), 'shuffle': True}
dataGeneratorVal = DataLoader(datasetVal, **params)
get_metrics(dataGeneratorVal)

{'0': {'precision': 0.999888706697631, 'recall': 0.9937102280687524, 'f1-score': 0.9967898933416175, 'support': 135617}, '1': {'precision': 0.8366526235158943, 'recall': 0.9965784671532847, 'f1-score': 0.9096398084530501, 'support': 4384}, 'accuracy': 0.9938000442853979, 'macro avg': {'precision': 0.9182706651067627, 'recall': 0.9951443476110186, 'f1-score': 0.9532148508973338, 'support': 140001}, 'weighted avg': {'precision': 0.9947771218613174, 'recall': 0.9938000442853979, 'f1-score': 0.9940608701764154, 'support': 140001}}
Precision: 0.9182706651067627, Recall: 0.9951443476110186, F1-Score: 0.9532148508973338, Accuracy: 0.9938000442853979, AccuracyCust: 0.9938000442853979


In [14]:
#do this one
params = {'batch_size': datasetTest.__len__(), 'shuffle': True}
dataGeneratorTest = DataLoader(datasetTest, **params)
get_metrics(dataGeneratorTest)

{'0': {'precision': 0.9998960607005509, 'recall': 0.9939043289596032, 'f1-score': 0.9968911917098446, 'support': 135506}, '1': {'precision': 0.8443271767810027, 'recall': 0.9968847352024922, 'f1-score': 0.9142857142857144, 'support': 4494}, 'accuracy': 0.994, 'macro avg': {'precision': 0.9221116187407767, 'recall': 0.9953945320810478, 'f1-score': 0.9555884529977795, 'support': 140000}, 'weighted avg': {'precision': 0.9949022995267334, 'recall': 0.994, 'f1-score': 0.99423955588453, 'support': 140000}}
Precision: 0.9221116187407767, Recall: 0.9953945320810478, F1-Score: 0.9555884529977795, Accuracy: 0.994, AccuracyCust: 0.994


In [17]:
torch.save(model, "tcn_unsw.pth")